# CESI HumanForYou

## Préparation de l'environnement

## Imports des librairies


In [ ]:
# imports
import numpy as np
import os
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import default_rng
from datetime import datetime

Affichage des graphs

In [ ]:
# stabilité du notebook d'une exécution à l'autre
random=default_rng(42) 

# jolies figures directement dans le notebook
plt.rcParams['figure.figsize'] = [15,10] 
plt.rcParams["figure.dpi"] = 250
plt.rcParams['axes.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

# où sauver les figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images") # le dossier doit exister

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings
warnings.filterwarnings('ignore')

## Import des données

In [ ]:
#Import des données depuis Github de Anthony Lorendeaux
general_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/general_data.csv"
manager_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/manager_survey_data.csv"
employee_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/employee_survey_data.csv"
in_time_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/in_time.csv"
out_time_url = "https://raw.githubusercontent.com/anthonylorendeaux/CESI-IA-CSV/master/out_time.csv"

#Lecture des csv
general_info_data = pd.read_csv(general_url)
manager_survey_data = pd.read_csv(manager_url)
employee_survey_data = pd.read_csv(employee_url)
in_time_data = pd.read_csv(in_time_url)
out_time_data = pd.read_csv(out_time_url)

## Visualisation des données
Dans un premier temps, nous pouvons observer les premières lignes de notre dataset en utilisant la méthode **head()**, afin de vérifier que l'import se soit bien passé.

In [ ]:
general_info_data.head(1)

Nous pouvons obtenir une description du dataset avec la méthode **info()**. 
Ce qui nous permettra de repérer les attributs contenant des données vides et de connaitre le type des valeurs de chaque attribut.

La fonction **shape()** quand à elle nous permet combien de ligne et de colonne nous avons dans notre csv.

In [ ]:
print('Shape of in_time :',general_info_data.shape)
general_info_data.info()

In [ ]:
manager_survey_data.head(1)

In [ ]:
print('Shape of in_time :',manager_survey_data.shape)
manager_survey_data.info()

In [ ]:
employee_survey_data.head(1)

In [ ]:
print('Shape of in_time :',employee_survey_data.shape)
employee_survey_data.info()

In [ ]:
in_time_data.head(1)

In [ ]:
print('Shape of in_time :',in_time_data.shape)
in_time_data.info()

In [ ]:
out_time_data.head(1)

In [ ]:
print('Shape of in_time :',out_time_data.shape)
out_time_data.info()

## Transformation des données

Les données de temps ne sont pas exploitables dans l'état actuel des choses, il faut donc les retravailler.
Les ddonnées sont pas expoitables il faut les tranformer.

Avoir des heures d'entrées et de sortie de nos employés n'est pas trop significatifs, c'est pour cela que l' on va remplacer tous les valeurs par la moyenne de temps de travail de chaque employé.

Mais : 
Les dates sont stockées en tant que chaine de caractère et c'est compliqué de les exploiter.
Certaines données valent (NaN), ce qui veut dire qu'un employé a été absent au travail
Il faut les transformer en objet datetime.

Lorsqu'un employé est absent au travail, son temps moyen de travail est de 0 donc on peut remplacer les NaN par 0.

D'abord on va rename la colonne Unnamed qui correspond à nos ID de nos employés puisque dans csv on a le même nombre de ligne que sur les autres csv.

In [ ]:
#On rename nos colonnes Unnamed de nos csv in et out
in_time_data.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
in_time_data.set_index('EmployeeID', inplace=True)
in_time_data
out_time_data.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
out_time_data.set_index('EmployeeID', inplace=True)

# On enlève les colonnes de vacances(là où il y a que des NaN) et donc là ou l'employé a été absent
in_time_data=in_time_data.dropna(axis=1,how='all')
out_time_data=out_time_data.dropna(axis=1,how='all')
#On remplace les NaN par 0
in_time_data.fillna(0, inplace=True)
out_time_data.fillna(0, inplace=True)

out_time_data.head()


On transforme nos chaines de caractère en objets datetime

In [ ]:
for date in in_time_data.columns:
    in_time_data[date]=pd.to_datetime(in_time_data[date])
    out_time_data[date]=pd.to_datetime(out_time_data[date])

On calcule dans un nouveau dataset le nombre d'heure passée au travail d'un employé par jour

In [ ]:
time_work_per_day=pd.DataFrame()

cols=in_time_data.columns
for col in cols:
    time_work_per_day[col]=((out_time_data[col] - in_time_data[col]).dt.total_seconds() /3600)

time_work_per_day.head()

On ajoute ensuite une colonne pour avoir la moyenne de temps passsé au travail par employé sur l'année 2015 et le nombre d'absences au travail par employé durant l'année 2015 et on garde que nos colonnes de la moyenne et des absences sur le dataset

In [ ]:
time_work_per_day['MeanTimeWorkOverYear2015']=round(time_work_per_day.astype(int).mean(axis=1),2)
time_work_per_day['absences_par_jour']=(time_work_per_day == 0).astype(int).sum(axis=1)
time_work_per_day = time_work_per_day.drop((time_work_per_day.columns[0:-2]), axis = 1)
time_work_per_day.head()

## Concaténation des données

In [ ]:
concat_time_csv = general_info_data.merge(time_work_per_day, on='EmployeeID')
concat_manager_csv = concat_time_csv.merge(manager_survey_data, on='EmployeeID')
temp_concat = concat_manager_csv.merge(employee_survey_data, on='EmployeeID')
temp_concat = temp_concat.set_index('EmployeeID')
temp_concat.describe().T

In [ ]:
final_data = temp_concat.copy()
final_data[final_data.columns[final_data.isnull().any()]].isnull().sum()

In [ ]:
final_data.fillna(round(final_data.median()),inplace=True)
final_data.describe().T

## Histogramme 

In [ ]:
final_data.hist(bins=50, figsize=(20,15))
plt.show()

## Normalisation

In [ ]:
# One hot
final_data2 = pd.get_dummies(final_data, 
prefix=["BusinessTravel","Department","EducationField", "JobRole","MaritalStatus", "Gender"],
columns=["BusinessTravel","Department","EducationField", "JobRole","MaritalStatus", "Gender"])
final_data2.head(5)



In [ ]:
# Delete colums
final_data2.drop(['EmployeeCount', 'Over18', 'StandardHours'], axis=1, inplace = True)
final_data2.head(5)

## Statistiques pour l'attrition

In [ ]:
cols = ['BusinessTravel', 'Department', 'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus', 'StockOptionLevel', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']

plt.figure()
for i, col in enumerate(cols):
    ax = plt.subplot(7, 2, i+1)
    plots = sns.histplot(data=final_data, x=col, ax=ax, hue=final_data['Attrition'], multiple='dodge', shrink=0.8)
    # for bar in plots.patches:
    #     percentage = '{:.1f}%'.format(100 * bar.get_height() / len(final_data[col]))
    #     x = bar.get_x() + bar.get_width() / 2
    #     y = bar.get_height()
    #     plots.annotate(percentage, (x, y),ha='center',va='center',size=15, xytext=(0, 3),
    #                textcoords='offset points')
# plt.tight_layout(pad=3.0)
plt.show()


In [ ]:
# Matrice de corrélation
corr_matrix = final_data2.corr(method="pearson")

plt.figure(figsize=(20,16))
sns.heatmap(corr_matrix, cbar=True)
plt.show()